In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd

In [29]:
df = pd.read_csv('/content/drive/MyDrive/Универ/Hotel_Reviews.csv', sep=',')
df

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515733,Wurzbachgasse 21 15 Rudolfsheim F nfhaus 1150 ...,168,8/30/2015,8.1,Atlantis Hotel Vienna,Kuwait,no trolly or staff to help you take the lugga...,14,2823,location,2,8,7.0,"[' Leisure trip ', ' Family with older childre...",704 day,48.203745,16.335677
515734,Wurzbachgasse 21 15 Rudolfsheim F nfhaus 1150 ...,168,8/22/2015,8.1,Atlantis Hotel Vienna,Estonia,The hotel looks like 3 but surely not 4,11,2823,Breakfast was ok and we got earlier check in,11,12,5.8,"[' Leisure trip ', ' Family with young childre...",712 day,48.203745,16.335677
515735,Wurzbachgasse 21 15 Rudolfsheim F nfhaus 1150 ...,168,8/19/2015,8.1,Atlantis Hotel Vienna,Egypt,The ac was useless It was a hot week in vienn...,19,2823,No Positive,0,3,2.5,"[' Leisure trip ', ' Family with older childre...",715 day,48.203745,16.335677
515736,Wurzbachgasse 21 15 Rudolfsheim F nfhaus 1150 ...,168,8/17/2015,8.1,Atlantis Hotel Vienna,Mexico,No Negative,0,2823,The rooms are enormous and really comfortable...,25,3,8.8,"[' Leisure trip ', ' Group ', ' Standard Tripl...",717 day,48.203745,16.335677


In [ ]:
# Выделяем из адреса отелей их страны
df["Hotel_Country"] = df["Hotel_Address"].map(lambda s: s.split()[-1])

In [31]:
# учитываем нюанс с отелем Регина, который имеется в нескольких странах под одинаковым названием
for i, row in df.iterrows():
  hn = row["Hotel_Name"]
  if hn == "Hotel Regina":
    df.at[i, "Hotel_Name"] = "%s (%s)" % (hn, row["Hotel_Country"])

In [32]:
# не просто королевство, а соединенное
df["Hotel_Country"] = df["Hotel_Country"].map(lambda s: "United Kingdom" if s == "Kingdom" else s)

In [40]:
# выделяем год отзыва из даты
df["Review_Year"] = df["Review_Date"].map(lambda s: s[-4:])

In [79]:
# выделяем год и месяц отзыва
df["Review_Month"] = df["Review_Date"].map(lambda s: "%s-%s" % (s[-4:], str(int(s.split('/')[0])).rjust(2,'0')))

In [ ]:
# рассчитываем так называемый средневзвешенный фидбек от посетителя, который учитывает его оценку непосредственно, общее количество отзывов от этого пользователя, его актуальность по времени и длину текста отзыва (формула разработана т. Аршиновым по наитию)
df["Review_Weighted_Score"] = df.apply(lambda x: (float(x["Reviewer_Score"])-5)*x["Total_Number_of_Reviews_Reviewer_Has_Given"]/(1+int(x["days_since_review"].replace(" days","").replace(" day","")))*(x["Review_Total_Positive_Word_Counts"]+x["Review_Total_Negative_Word_Counts"]), axis=1)

In [97]:
# чистим отзывы от пользователей с неопознанной национальностью
df = df.drop(df[(df['Reviewer_Nationality'].str.strip() == "")|(df['Reviewer_Nationality'] is None)].index)

In [99]:
# ну и напоследок переводим дату в формат здорового человека
df["Review_Date"] = df["Review_Date"].map(lambda s: "%s-%s-%s" % (s[-4:], str(int(s.split('/')[0])).rjust(2,'0'), str(int(s.split('/')[1])).rjust(2,'0')))

In [100]:
df.to_csv("/content/drive/MyDrive/Универ/data.csv", encoding='utf-8', index=False)